In [53]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import HTML
import numpy as np

plt.ioff();

In [134]:
DATA_URL = 'https://raw.githubusercontent.com/splotniks/inequal4/main/GCIPrawdata.csv'


EXPLANATION = ""
"""\
<div class="app-sidebar">
<p><em>Comparisons within a country</em><p>

<p>The 90:10 ratio measures the 9th decile of income over the 1st. The larger this number is, the more unequal the income share between these groups is.

...

</p> The Gini will be discussed later on, but provides an equality measure that factors in all income deciles. </p>
"""

'<div class="app-sidebar">\n<p><em>Comparisons within a country</em><p>\n\n<p>The 90:10 ratio measures the 9th decile of income over the 1st. The larger this number is, the more unequal the income share between these groups is.\n\n...\n\n</p> The Gini will be discussed later on, but provides an equality measure that factors in all income deciles. </p>\n'

In [55]:
HTML("""\
<style>
.app-subtitle {
    font-size: 1.5em;
}

.app-subtitle a {
    color: #106ba3;
}

.app-subtitle a:hover {
    text-decoration: underline;
}

.app-sidebar p {
    margin-bottom: 1em;
    line-height: 1.7;
}

.app-sidebar a {
    color: #106ba3;
}

.app-sidebar a:hover {
    text-decoration: underline;
}
</style>
""")

In [104]:
def GiniArea(H): 
    H = np.sort(H)
    H_total = sum(H)
    cumul_H = np.cumsum(H).tolist() 
    portions_H=[0]
    for i in cumul_H:
        portions_H.append(cumul_H[cumul_H.index(i)]/H_total*100) 
    B = np.trapz(portions_H, dx = 1/len(H))
    GiniApprox = ((50-B)/50)

    return (GiniApprox)

In [141]:
class App:
#version1
    
    def __init__(self, df):
        self._df = df
        available_indicators = self._df['Country'].unique()
        available_metrics = ['Income ratio 50:10', 'Income ratio 90:50', 'Income ratio 90:10', 'Gini']
        self._x_dropdown = self._create_indicator_dropdown(available_indicators, -7)
        self._y_dropdown = self._create_indicator_dropdown(available_metrics, 0)  
        self._plot_container = widgets.Output()
        self._year_slider, year_slider_box = self._create_year_slider(
            min(df['Year']), max(df['Year'])
        )

        _app_container = widgets.VBox([
            widgets.HBox([self._x_dropdown, self._y_dropdown]),
            self._plot_container,
            year_slider_box
        ], layout=widgets.Layout(align_items='center', flex='3 0 auto'))
        self.container = widgets.VBox([
            widgets.HTML(
                (
                    """<h1>Simple Inequality Measures (Option 1)</h1>
                    <p><em>Comparisons within a country</em><p>

<p>The 90:10 ratio measures the 9th decile of income over the 1st. The larger this number is, the more unequal the income share between these groups is.

</p> The Gini will be discussed later on, but provides an equality measure that factors in all income deciles.
"""          
                ), 
                layout=widgets.Layout(margin='0 0 5em 0')
            ),
            widgets.HBox([
                _app_container, 
                widgets.HTML(EXPLANATION, layout=widgets.Layout(margin='0 0 0 2em'))
            ])
        ], layout=widgets.Layout(flex='1 1 auto', margin='0 auto 0 auto', max_width='1024px'))
        self._update_app()     
        
    @classmethod
    def from_url(cls, url):
        df = pd.read_csv(url)
        #df['Gini'] = GiniArea(df.iloc[:, 2:12])
        X = []
        for x in range (0,len(df)):
            X.append(GiniArea(np.array(df.iloc[x, 2:12]).astype(np.double)))
        df['Gini'] = X
        #df['Gini'] = GiniArea(np.array(df.iloc[1:, 2:12]).astype(np.double))
        df['Income ratio 50:10'] = df['Decile 5 Income']/df['Decile 1 Income']
        df['Income ratio 90:50'] = df['Decile 9 Income']/df['Decile 5 Income']
        df['Income ratio 90:10'] = df['Decile 9 Income']/df['Decile 1 Income']
        return cls(df)
        
    def _create_indicator_dropdown(self, indicators, initial_index):
        dropdown = widgets.Dropdown(options=indicators, value=indicators[initial_index])
        dropdown.observe(self._on_change, names=['value'])
        return dropdown
    
    def _create_year_slider(self, min_year, max_year):
        year_slider_label = widgets.Label('Year range: ')
        year_slider = widgets.IntRangeSlider(
            min=min_year, max=max_year,
            layout=widgets.Layout(width='500px')
        )
        year_slider.observe(self._on_change, names=['value'])
        year_slider_box = widgets.HBox([year_slider_label, year_slider])
        return year_slider, year_slider_box
    
    def _create_plot(self, x_indicator, y_indicator, year_range):
        df = self._df[self._df['Year'].between(*year_range)]
        xs = df[df['Country'] == x_indicator][y_indicator]
        years = np.arange(year_range[0], year_range[1]+1,1)
        ys = df[df['Country'] == y_indicator]['Decile 1 Income']
        plt.figure(figsize=(14, 8))
        plt.xlabel(x_indicator, size=16)
        plt.ylabel(y_indicator, size=16)
        plt.gca().tick_params(axis='both', which='major', labelsize=16)
        plt.bar(years, xs)
        
    def _on_change(self, _):
        self._update_app()
        
    def _update_app(self):
        x_indicator = self._x_dropdown.value
        y_indicator = self._y_dropdown.value
        year_range = self._year_slider.value
        self._plot_container.clear_output(wait=True)
        with self._plot_container:
            self._create_plot(x_indicator, y_indicator, year_range)
            plt.show()


In [142]:
app = App.from_url(DATA_URL)

app.container

In [151]:
class App2:
#version2
    
    def __init__(self, df):
        self._df = df
        available_indicators = self._df['Country'].unique()
        available_metrics = ['Income ratio 50:10', 'Income ratio 90:50', 'Income ratio 90:10', 'Gini']
        self._x_dropdown = self._create_indicator_dropdown(available_indicators, -7)
        self._y_dropdown = self._create_indicator_dropdown(available_indicators, 0)  
        self._a_dropdown = self._create_indicator_dropdown(available_metrics, 0) 
        self._plot_container = widgets.Output()
        self._year_slider, year_slider_box = self._create_year_slider(
            min(df['Year']), max(df['Year'])
        )

        _app_container = widgets.VBox([
            widgets.HBox([self._a_dropdown,self._x_dropdown, self._y_dropdown]),
            self._plot_container,
            year_slider_box
        ], layout=widgets.Layout(align_items='center', flex='3 0 auto'))
        self.container = widgets.VBox([
            widgets.HTML(
                (
                    """<h1>Simple Inequality Measures (Option 2)</h1>
                    <p><em>Comparisons between countries</em><p>

<p>The 90:10 ratio measures the 9th decile of income over the 1st. The larger this number is, the more unequal the income share between these groups is.

</p> The Gini will be discussed later on, but provides an equality measure that factors in all income deciles.
"""          
                ), 
                layout=widgets.Layout(margin='0 0 5em 0')
            ),
            widgets.HBox([
                _app_container, 
                widgets.HTML(EXPLANATION, layout=widgets.Layout(margin='0 0 0 2em'))
            ])
        ], layout=widgets.Layout(flex='1 1 auto', margin='0 auto 0 auto', max_width='1024px'))
        self._update_app()     
        
    @classmethod
    def from_url(cls, url):
        df = pd.read_csv(url)
        X = []
        for x in range (0,len(df)):
            X.append(GiniArea(np.array(df.iloc[x, 2:12]).astype(np.double)))
        df['Gini'] = X
        df['Income ratio 50:10'] = df['Decile 5 Income']/df['Decile 1 Income']
        df['Income ratio 90:50'] = df['Decile 9 Income']/df['Decile 5 Income']
        df['Income ratio 90:10'] = df['Decile 9 Income']/df['Decile 1 Income']
        return cls(df)
        
    def _create_indicator_dropdown(self, indicators, initial_index):
        dropdown = widgets.Dropdown(options=indicators, value=indicators[initial_index])
        dropdown.observe(self._on_change, names=['value'])
        return dropdown
    
    def _create_year_slider(self, min_year, max_year):
        year_slider_label = widgets.Label('Year range: ')
        year_slider = widgets.IntRangeSlider(
            min=min_year, max=max_year,
            layout=widgets.Layout(width='500px')
        )
        year_slider.observe(self._on_change, names=['value'])
        year_slider_box = widgets.HBox([year_slider_label, year_slider])
        return year_slider, year_slider_box
    
    def _create_plot(self, a_indicator, x_indicator, y_indicator, year_range):
        df = self._df[self._df['Year'].between(*year_range)]
        xs = df[df['Country'] == x_indicator][a_indicator]
        years = np.arange(year_range[0], year_range[1]+1,1)
        ys = df[df['Country'] == y_indicator][a_indicator]
        #ys = df[df['Country'] == y_indicator]['Decile 1 Income']
        plt.figure(figsize=(14, 8))
        plt.xlabel(x_indicator, size=16)
        plt.ylabel(y_indicator, size=16)
        plt.gca().tick_params(axis='both', which='major', labelsize=16)
        plt.plot(years, xs, label = x_indicator)
        plt.plot(years, ys, label = y_indicator)
        plt.legend()
        
    def _on_change(self, _):
        self._update_app()
        
    def _update_app(self):
        x_indicator = self._x_dropdown.value
        y_indicator = self._y_dropdown.value
        a_indicator = self._a_dropdown.value
        year_range = self._year_slider.value
        self._plot_container.clear_output(wait=True)
        with self._plot_container:
            self._create_plot(a_indicator,x_indicator, y_indicator, year_range)
            plt.show()


In [152]:
app2 = App2.from_url(DATA_URL)

app2.container